# Train RetinaNet on AFT dataset

This notebook shows how train a RetinaNet model on the American Football Tactical (AFT) dataset using `detectron2`.

## Get the AFT dataset from `quilt3`

Configure `quilt3` to look into our server and registry:

In [ ]:
import quilt3
import pandas as pd


# Commented lines only necessary the first time
# quilt3.config('https://quilt3.hudltools.com/')
# quilt3.login() 
quilt3.config(default_remote_registry='s3://hudlrd-datasets') 

Get the AFT dataset from `quilt3`:

In [ ]:
user = 'hudlrd'
package = 'american_football_tactical'
parquet = 'detections_df.parquet'

pkg = quilt3.Package.install(
    f'{user}/{package}', 
    registry=None,
    top_hash=None
)

pkg[parquet].fetch()
df = pd.read_parquet(parquet)

In [ ]:
print("# examples: ", len(df))

df

Clean up the dataframe:

In [ ]:
# Remove examples tagged as not match frames
aux_df = df
df = df[df.notMatchFrame == False]
print('# examples removed by notMatchFrame filtering: ', len(aux_df) - len(df))

# Remove examples with null boxes or labels
aux_df = df
df = df[~df.bbxs.isnull()]
df = df[~df.labels.isnull()]
print('# examples removed by isnull filtering: ', len(aux_df) - len(df))

# Remove examples with no boxes or labels
aux_df = df
df = df[df.bbxs.map(lambda d: len(d)) > 0]
df = df[df.labels.map(lambda d: len(d)) > 0]
print('# examples removed by empty box or label list filtering: ', len(aux_df) - len(df))

# Remove invalid bounding boxes where width or height are smaller or equak than 
aux_df = df
df = df[df.bbxs.map(lambda bbxs: len(set([True for b in bbxs if b[2] <= 0 or b[3] <= 0]))) == 0]
print('# examples removed by invalid box filtering: ', len(aux_df) - len(df))

# Reset dataframe indices
df = df.reset_index(drop=True)

In [ ]:
print('# examples: ', len(df))

df

At this point is probably a good idea to save the dataset's images to the local machine and update the data frame accordingly:

In [ ]:
import shutil
import requests
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor


cache = Path('./aft_images/')
cache.mkdir(parents=True, exist_ok=True)


def save_df_images(df):
    def save_df_image(i):
        local_path = cache / f"{str(i).zfill(4)}.png"
        if not local_path.exists():
            with open(local_path, 'wb') as f:
                response = requests.get(df.at[i, 'path'], stream=True)
                shutil.copyfileobj(response.raw, f)
        df.at[i, 'path'] = str(local_path.absolute())

    with ThreadPoolExecutor() as executor:
        list(executor.map(save_df_image, range(len(df))))

In [ ]:
save_df_images(df)

In [ ]:
df

## Register the AFT dataset to `detectron2`

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

Get the dataset's unique labels:

In [ ]:
unique_labels = set()

for labels in df["labels"]:
    if labels is not None:
        unique_labels.update(labels)

unique_labels = list(unique_labels)
        
print("Unique labels:", unique_labels)

Define the `get_dicts` function that will return the items in our dataset in the format expected by `detectron2`:

In [ ]:
import numpy as np
from detectron2.structures import BoxMode


def get_dicts(df, partition):
    df = df[df.set_split_random == partition]
    
    records = []
    for i, row in df.iterrows():
        record = {}
        record["file_name"] = row['path']
        record["image_id"] = i
        record["height"] = 2000
        record["width"] = 2666

        annotations = []
        for bbox, label in zip(row["bbxs"], row["labels"]):
            ann = {}
            bbox = bbox * np.asanyarray([record["width"], record["height"]] * 2)
            ann["bbox"] = bbox.tolist()
            ann["bbox_mode"] = BoxMode.XYWH_ABS
            ann["category_id"] = unique_labels.index(label)
            annotations.append(ann)

        record["annotations"] =  annotations
        records.append(record)
    
    return records

In [ ]:
print("# examples in train:", len(get_dicts(df, "train")))
print("# examples in dev:", len(get_dicts(df, "dev")))
print("# examples in test:", len(get_dicts(df, "test")))

Tell `detectron2` about the previous function:

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog


for partition in ["train", "dev", "test"]:
    DatasetCatalog.register(
        f"AmericanFootballTactical/{partition}", 
        lambda partition = partition: get_dicts(df, partition)
    )
    MetadataCatalog.get(
        f"AmericanFootballTactical/{partition}"
    ).set(thing_classes=unique_labels)

Make sure the dataset has been correctly registered:

In [ ]:
# DatasetCatalog.get('AmericanFootballTactical/train')

Finally, let us plot some of the dataset's data:

In [ ]:
# %matplotlib inline
# import matplotlib.pyplot as plt

# import random
# import requests
# from PIL import Image
# from detectron2.utils.visualizer import Visualizer


# figsize = (16, 12)

# metadata = MetadataCatalog.get("AmericanFootballTactical/train")
# records = get_dicts(df, "train")

# for i, r in enumerate(random.sample(records, 5)):
# #     img = Image.open(requests.get(r["file_name"], stream=True).raw)
#     img = Image.open(r["file_name"])
#     img = np.asanyarray(img)
    
#     visualizer = Visualizer(img, metadata=metadata, scale=0.5)
#     vis = visualizer.draw_dataset_dict(r)
    
#     plt.figure(i, figsize=figsize)
#     plt.imshow(vis.get_image())

## Train RetinaNet

In [ ]:
from detectron2.engine import DefaultTrainer
from detectron2.config import get_cfg

In [ ]:
cfg = get_cfg()

In [ ]:
print(cfg)

In [ ]:
cfg.merge_from_file("../../../configs/COCO-Detection/retinanet_R_50_FPN_3x.yaml")

In [ ]:
cfg.DATASETS.TRAIN = ("AmericanFootballTactical/train",)
cfg.DATASETS.TEST = ()
cfg.SOLVER.IMS_PER_BATCH = 6
cfg.SOLVER.MAX_ITER = 10000

In [ ]:
cfg.SOLVER.BASE_LR = 3e-3

In [ ]:
import os


os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)

In [ ]:
trainer = DefaultTrainer(cfg) 

In [ ]:
trainer.resume_or_load(resume=True)

In [ ]:
trainer.train()

## Evaluate RetinaNet

In [ ]:
from pathlib import Path
from detectron2.engine import DefaultPredictor


cfg.MODEL.WEIGHTS = str(Path(cfg.OUTPUT_DIR) / "model_final.pth")
cfg.DATASETS.TEST = ("AmericanFootballTactical/dev","AmericanFootballTactical/test")

predictor = DefaultPredictor(cfg)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import random
from PIL import Image
from detectron2.utils.visualizer import Visualizer


figsize = (16, 12)

records = get_dicts(df, partition="dev")
metadata = MetadataCatalog.get("AmericanFootballTactical/dev")

for i, r in enumerate(random.sample(records, 3)):
    img = np.asanyarray(Image.open(r["file_name"]))
    
    outputs = predictor(img)
    
    v = Visualizer(
        img,
        metadata=metadata,
        scale=0.5,
    )
    v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
    
    plt.figure(i, figsize=figsize)
    plt.imshow(v.get_image())